# Imports

In [1]:
import pandas as pd
import os
from glob import glob
from typing import List

# Files

In [6]:
src_path = os.path.join("/", "Volumes", "owen", "datas", "rplace", "processed")
src_glob = os.path.join(src_path, "2022_place_canvas_history-*.csv")
dest_path = os.path.join(src_path, "..")

In [7]:
files = glob(src_glob)
files

['/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000000.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000001.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000002.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000003.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000004.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000005.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000006.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000007.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000008.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000009.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000010.csv',
 '/Volumes/owen/datas/rplace/processed/2022_place_canvas_history-00000000001

# Data processing

In [14]:
def count_hours(filename: str):
    df_place = pd.read_csv(filename, index_col=0)
    df_place["date"] = pd.to_datetime(df_place["date"])
    df_place = df_place.resample("1h", on="date").count()

    df_place["hour"] = df_place.index.hour
    df_place["day"] = df_place.index.day
    df_place["month"] = df_place.index.month
    df_place["year"] = df_place.index.year

    df_place.set_index(["day", "month", "year", "hour"], inplace=True)

    return df_place[["user_id"]].rename({"user_id": "count"}, axis=1).sort_index()

In [15]:
count_hours(filename=files[0])

/Users/owen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


count
day month year hour         
4   4     2022 0      140799
               1     1034518

In [18]:
def merge_hours_count(files: List[str]):
    df: pd.DataFrame = count_hours(files[0])

    for i in range(1, len(files)):
        print(f"Adding {files[i]} ({i / len(files) * 100}%)")
        df_to_add = count_hours(files[i])
        df = df.add(df_to_add, fill_value=1)
    
    return df.sort_index()

In [20]:
df = merge_hours_count(files[:])
df

/Users/owen/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000001.csv (1.2658227848101267%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000002.csv (2.5316455696202533%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000003.csv (3.79746835443038%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000004.csv (5.063291139240507%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000005.csv (6.329113924050633%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000006.csv (7.59493670886076%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000007.csv (8.860759493670885%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000008.csv (10.126582278481013%)
Adding /Volumes/owen/datas/rplace/processed/2022_place_canvas_history-000000000009.csv (11.39240506329114%)
Adding /Volumes/owen/datas/

count
day month year hour           
1   4     2022 12         88.0
               13     548837.0
               14     867910.0
               15     996994.0
               16    1070997.0
...                        ...
4   4     2022 20    4904543.0
               21    5901541.0
               22    5640544.0
               23    3057405.0
5   4     2022 0      320011.0

[85 rows x 1 columns]

In [21]:
df.to_csv(os.path.join(dest_path, "hours_count2.csv"))